In [2]:
import itertools
from pprint import pprint
from qiskit import QuantumCircuit, Aer, execute
backend = Aer.get_backend('qasm_simulator')  

maxgc = 4
gs = ['x','ccx']
qbnos = 4
qb = list(range(0,qbnos))

In [3]:
qb

[0, 1, 2, 3]

In [4]:
def toStr(n,base):
    '''
    Convert a decimal number to base-n
    '''
    convertString = "0123456789ABCDEF"
    if n < base:
        return convertString[n]
    else:
        return toStr(n//base,base) + convertString[n%base]

In [5]:
toStr(15,16)

'F'

In [6]:
def X_choice(opcode):
    c = list(itertools.combinations(qb, 1))
    #print(c)
    allc = []
    for i in c:
        allc.append(opcode+str(i[0]))
    return allc

In [7]:
def CCX_choice(opcode):
    c = list(itertools.combinations(qb, 3))
    allc = []
    #print(c)
    for i in c:
        allc.append(opcode+str(i[0])+str(i[1])+str(i[2]))
        allc.append(opcode+str(i[1])+str(i[2])+str(i[0]))
        allc.append(opcode+str(i[2])+str(i[0])+str(i[1]))
    return allc

In [8]:
setX = X_choice('0')
setCCX = CCX_choice('1')

In [9]:
setX

['00', '01', '02', '03']

In [10]:
setCCX

['1012',
 '1120',
 '1201',
 '1013',
 '1130',
 '1301',
 '1023',
 '1230',
 '1302',
 '1123',
 '1231',
 '1312']

In [11]:
def progs(gc):
    p = []
    for i in range(0,len(gs)**gc):
        gseq = toStr(i,len(gs)).zfill(gc)
        #print(gseq)
        cg = ['']          
        for j in range(0, len(gseq)):
            if gseq[j] == '0':
                g = list(itertools.product(cg, setX))
            if gseq[j] == '1':
                g = list(itertools.product(cg, setCCX))
            cg = []
            for k in g:
                cg.append(''.join(map(str, k)))
        #print(gseq,': ',cg)
        for j in cg:
            p.append(j)
    return p

In [13]:
def state_construction(index,qc):
    for i in range(4):
        if(index[i]) == '1':
            qc.x(i)

In [14]:
possible_states = list(itertools.product('01', repeat=4))
possible_states

[('0', '0', '0', '0'),
 ('0', '0', '0', '1'),
 ('0', '0', '1', '0'),
 ('0', '0', '1', '1'),
 ('0', '1', '0', '0'),
 ('0', '1', '0', '1'),
 ('0', '1', '1', '0'),
 ('0', '1', '1', '1'),
 ('1', '0', '0', '0'),
 ('1', '0', '0', '1'),
 ('1', '0', '1', '0'),
 ('1', '0', '1', '1'),
 ('1', '1', '0', '0'),
 ('1', '1', '0', '1'),
 ('1', '1', '1', '0'),
 ('1', '1', '1', '1')]

In [15]:
def conv2QP(desc):
    qc_list = []
    for j in possible_states:
        qcirc = QuantumCircuit(qbnos,qbnos)
        state_construction(j,qcirc)
        i = 0
        while (i < len(desc)):
            if desc[i]=='0':
                # print('X',desc[i+1])
                qcirc.x(int(desc[i+1]))
                i+= 2
            elif desc[i]=='1':
                # print('CCX',desc[i+1],desc[i+2],desc[i+3])
                qcirc.ccx(int(desc[i+1]),int(desc[i+2]),int(desc[i+3]))
                i+= 4
        qcirc.barrier()
        for i in range(0,qbnos):
            qcirc.measure(i,i)
        qc_list.append(qcirc)
    return qc_list

In [16]:
def runQprog(desc):
    mem_result= []
    qcirc = conv2QP(desc)
    # print(desc)
    # print(qcirc.draw())
    for i in qcirc:
        job = execute(i, backend, shots=1, memory=True)
        result = job.result()
        memory = result.get_memory()
        mem_result.append(memory[0][::-1])
    return mem_result

In [30]:
runQprog('00')

['1000',
 '1001',
 '1010',
 '1011',
 '1100',
 '1101',
 '1110',
 '1111',
 '0000',
 '0001',
 '0010',
 '0011',
 '0100',
 '0101',
 '0110',
 '0111']

In [18]:
gc = 1
ng = {}
p = progs(gc)
for i in p:
    op = runQprog(i)
    ng[i] = op
print("nGC Level",gc)
# for key, value in sorted(ng.items()):
#     value.sort(key=len)
#     print(key, len(value),value[0],len(value[0]))

nGC Level 1


In [19]:
one_gate = progs(1)

A -> B (layer 1)
A -> -> B (layer 2)

In [20]:
l = []
for i in (one_gate):
    l.append(ng[i])

In [21]:
l

[['1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '1110',
  '1111',
  '0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111'],
 ['0100',
  '0101',
  '0110',
  '0111',
  '0000',
  '0001',
  '0010',
  '0011',
  '1100',
  '1101',
  '1110',
  '1111',
  '1000',
  '1001',
  '1010',
  '1011'],
 ['0010',
  '0011',
  '0000',
  '0001',
  '0110',
  '0111',
  '0100',
  '0101',
  '1010',
  '1011',
  '1000',
  '1001',
  '1110',
  '1111',
  '1100',
  '1101'],
 ['0001',
  '0000',
  '0011',
  '0010',
  '0101',
  '0100',
  '0111',
  '0110',
  '1001',
  '1000',
  '1011',
  '1010',
  '1101',
  '1100',
  '1111',
  '1110'],
 ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1110',
  '1111',
  '1100',
  '1101'],
 ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '1110',
  '1111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '0110',
  '0111'],
 ['0000',
  '0001',
  '0010',
  '0

In [32]:
import numpy as np

In [33]:
possible_states = ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '1110',
  '1111']

In [34]:
# short circuits at shortest nested list if table is jagged:
list(map(list, zip(*l)))

# discards no data if jagged and fills short nested lists with None
transpose_list = list(map(list, itertools.zip_longest(*l, fillvalue=None)))

### List of all first layer 

In [36]:
transpose_list

[['1000',
  '0100',
  '0010',
  '0001',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000'],
 ['1001',
  '0101',
  '0011',
  '0000',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001'],
 ['1010',
  '0110',
  '0000',
  '0011',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010'],
 ['1011',
  '0111',
  '0001',
  '0010',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '1011',
  '0011',
  '0011',
  '0111',
  '0011'],
 ['1100',
  '0000',
  '0110',
  '0101',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100'],
 ['1101',
  '0001',
  '0111',
  '0100',
  '0101',
  '0101',
  '0101',
  '0101',
  '1101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0111'],
 ['1110',
  '0010',
  '0100',
  '0

In [25]:
all_first_layer_list={}
for i in range(qbnos**2):
    all_first_layer_list[possible_states[i]] = list(np.unique(transpose_list[i]))

In [26]:
all_first_layer_list

{'0000': ['0000', '0001', '0010', '0100', '1000'],
 '0001': ['0000', '0001', '0011', '0101', '1001'],
 '0010': ['0000', '0010', '0011', '0110', '1010'],
 '0011': ['0001', '0010', '0011', '0111', '1011'],
 '0100': ['0000', '0100', '0101', '0110', '1100'],
 '0101': ['0001', '0100', '0101', '0111', '1101'],
 '0110': ['0010', '0100', '0110', '0111', '1110'],
 '0111': ['0011', '0101', '0110', '0111', '1111'],
 '1000': ['0000', '1000', '1001', '1010', '1100'],
 '1001': ['0001', '1000', '1001', '1011', '1101'],
 '1010': ['0010', '1000', '1010', '1011', '1110'],
 '1011': ['0011', '1001', '1010', '1011', '1111'],
 '1100': ['0100', '1000', '1100', '1101', '1110'],
 '1101': ['0101', '1001', '1100', '1101', '1111'],
 '1110': ['0110', '1010', '1100', '1110', '1111'],
 '1111': ['0111', '1011', '1101', '1110']}

In [32]:
all_first_layer_list['0000']

['0000', '0001', '0010', '0100', '1000']

### Adjancency matrix with corresponding level

In [33]:
def create_adjacency_layer(state):
    layer = 1
    first_state_adjacency = {}
    for i in possible_states:
        first_state_adjacency[i] = 0
    for i in all_first_layer_list[state]:
        if((first_state_adjacency[i] == 0)):
            first_state_adjacency[i] = 1
    for j in possible_states:
        if((first_state_adjacency[j] == 1) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 2
    for j in possible_states:
        if((first_state_adjacency[j] == 2) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 3
    for j in possible_states:
        if((first_state_adjacency[j] == 3) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 4
    return first_state_adjacency

In [39]:
create_adjacency_layer('1110')

{'0000': 3,
 '0001': 4,
 '0010': 2,
 '0011': 3,
 '0100': 2,
 '0101': 3,
 '0110': 1,
 '0111': 2,
 '1000': 2,
 '1001': 3,
 '1010': 1,
 '1011': 2,
 '1100': 1,
 '1101': 2,
 '1110': 1,
 '1111': 1}

In [35]:
adjancency_matrix = []
for i in possible_states:
    adjancency_matrix.append(create_adjacency_layer(i))

In [38]:
adjancency_matrix[1]

{'0000': 1,
 '0001': 1,
 '0010': 2,
 '0011': 1,
 '0100': 2,
 '0101': 1,
 '0110': 3,
 '0111': 2,
 '1000': 2,
 '1001': 1,
 '1010': 3,
 '1011': 2,
 '1100': 3,
 '1101': 2,
 '1110': 4,
 '1111': 3}

### List first layer with number of program

In [37]:
transpose_list

[['1000',
  '0100',
  '0010',
  '0001',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000'],
 ['1001',
  '0101',
  '0011',
  '0000',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001'],
 ['1010',
  '0110',
  '0000',
  '0011',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010'],
 ['1011',
  '0111',
  '0001',
  '0010',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '1011',
  '0011',
  '0011',
  '0111',
  '0011'],
 ['1100',
  '0000',
  '0110',
  '0101',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100'],
 ['1101',
  '0001',
  '0111',
  '0100',
  '0101',
  '0101',
  '0101',
  '0101',
  '1101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0111'],
 ['1110',
  '0010',
  '0100',
  '0

In [53]:
def create_new_dict():
    new_dictionary = {}
    for i in possible_states:
        new_dictionary[i] = 0
    return new_dictionary

In [57]:
transpose_list

[['1000',
  '0100',
  '0010',
  '0001',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000',
  '0000'],
 ['1001',
  '0101',
  '0011',
  '0000',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001',
  '0001'],
 ['1010',
  '0110',
  '0000',
  '0011',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010',
  '0010'],
 ['1011',
  '0111',
  '0001',
  '0010',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '0011',
  '1011',
  '0011',
  '0011',
  '0111',
  '0011'],
 ['1100',
  '0000',
  '0110',
  '0101',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100',
  '0100'],
 ['1101',
  '0001',
  '0111',
  '0100',
  '0101',
  '0101',
  '0101',
  '0101',
  '1101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0101',
  '0111'],
 ['1110',
  '0010',
  '0100',
  '0

In [70]:
list_count = {}
k = 0
for i in transpose_list:
    new_dict = create_new_dict()
    for j in i:
         new_dict[j] += 1 
    list_count[possible_states[k]] = new_dict
    k+=1

In [83]:
list_count ### With length = 1

{'0000': {'0000': 12,
  '0001': 1,
  '0010': 1,
  '0011': 0,
  '0100': 1,
  '0101': 0,
  '0110': 0,
  '0111': 0,
  '1000': 1,
  '1001': 0,
  '1010': 0,
  '1011': 0,
  '1100': 0,
  '1101': 0,
  '1110': 0,
  '1111': 0},
 '0001': {'0000': 1,
  '0001': 12,
  '0010': 0,
  '0011': 1,
  '0100': 0,
  '0101': 1,
  '0110': 0,
  '0111': 0,
  '1000': 0,
  '1001': 1,
  '1010': 0,
  '1011': 0,
  '1100': 0,
  '1101': 0,
  '1110': 0,
  '1111': 0},
 '0010': {'0000': 1,
  '0001': 0,
  '0010': 12,
  '0011': 1,
  '0100': 0,
  '0101': 0,
  '0110': 1,
  '0111': 0,
  '1000': 0,
  '1001': 0,
  '1010': 1,
  '1011': 0,
  '1100': 0,
  '1101': 0,
  '1110': 0,
  '1111': 0},
 '0011': {'0000': 0,
  '0001': 1,
  '0010': 1,
  '0011': 10,
  '0100': 0,
  '0101': 0,
  '0110': 0,
  '0111': 2,
  '1000': 0,
  '1001': 0,
  '1010': 0,
  '1011': 2,
  '1100': 0,
  '1101': 0,
  '1110': 0,
  '1111': 0},
 '0100': {'0000': 1,
  '0001': 0,
  '0010': 0,
  '0011': 0,
  '0100': 12,
  '0101': 1,
  '0110': 1,
  '0111': 0,
  '1000': 0,
  

In [91]:
list_count_l2 = {}
for i in possible_states:
    sub_list_2 = {}
    for h in possible_states:
        sub_list_2[h] = 0
    for j in possible_states:
        if list_count[i][j] != 0 :
            for k in possible_states:
                if list_count[j][k] != 0:
                    sub_list_2[k] += list_count[j][k]*list_count[i][j]
                else:
                    sub_list_2[k] += list_count[i][k]
    list_count_l2[i] = sub_list_2

In [92]:
list_count_l2 # with length = 2

{'0000': {'0000': 148,
  '0001': 27,
  '0010': 27,
  '0011': 2,
  '0100': 27,
  '0101': 2,
  '0110': 2,
  '0111': 0,
  '1000': 27,
  '1001': 2,
  '1010': 2,
  '1011': 0,
  '1100': 2,
  '1101': 0,
  '1110': 0,
  '1111': 0},
 '0001': {'0000': 27,
  '0001': 148,
  '0010': 2,
  '0011': 25,
  '0100': 2,
  '0101': 25,
  '0110': 0,
  '0111': 4,
  '1000': 2,
  '1001': 25,
  '1010': 0,
  '1011': 4,
  '1100': 0,
  '1101': 4,
  '1110': 0,
  '1111': 0},
 '0010': {'0000': 27,
  '0001': 2,
  '0010': 148,
  '0011': 25,
  '0100': 2,
  '0101': 0,
  '0110': 25,
  '0111': 4,
  '1000': 2,
  '1001': 0,
  '1010': 25,
  '1011': 4,
  '1100': 0,
  '1101': 0,
  '1110': 4,
  '1111': 0},
 '0011': {'0000': 2,
  '0001': 25,
  '0010': 25,
  '0011': 110,
  '0100': 0,
  '0101': 5,
  '0110': 5,
  '0111': 38,
  '1000': 0,
  '1001': 5,
  '1010': 5,
  '1011': 38,
  '1100': 0,
  '1101': 0,
  '1110': 0,
  '1111': 16},
 '0100': {'0000': 27,
  '0001': 2,
  '0010': 2,
  '0011': 0,
  '0100': 148,
  '0101': 25,
  '0110': 25,
  '

In [104]:
list_count_l3 = {}
for i in possible_states:
    sub_list_3 = {}
    for h in possible_states:
        sub_list_3[h] = 0
    for j in possible_states:
        if list_count_l2[i][j] != 0 :
            for k in possible_states:
                if list_count[j][k] != 0:
                    sub_list_3[k] += list_count[j][k]*list_count_l2[i][j]
                else:
                    sub_list_3[k] += list_count_l2[i][k]
    list_count_l3[i] = sub_list_3

In [105]:
list_count_l3 # with length = 3

{'0000': {'0000': 2772,
  '0001': 640,
  '0010': 640,
  '0011': 90,
  '0100': 640,
  '0101': 90,
  '0110': 90,
  '0111': 12,
  '1000': 640,
  '1001': 90,
  '1010': 90,
  '1011': 12,
  '1100': 90,
  '1101': 12,
  '1110': 12,
  '1111': 0},
 '0001': {'0000': 640,
  '0001': 2766,
  '0010': 92,
  '0011': 566,
  '0100': 92,
  '0101': 566,
  '0110': 12,
  '0111': 156,
  '1000': 92,
  '1001': 566,
  '1010': 12,
  '1011': 156,
  '1100': 12,
  '1101': 156,
  '1110': 0,
  '1111': 48},
 '0010': {'0000': 640,
  '0001': 92,
  '0010': 2766,
  '0011': 566,
  '0100': 92,
  '0101': 12,
  '0110': 566,
  '0111': 156,
  '1000': 92,
  '1001': 12,
  '1010': 566,
  '1011': 156,
  '1100': 12,
  '1101': 0,
  '1110': 156,
  '1111': 48},
 '0011': {'0000': 90,
  '0001': 572,
  '0010': 572,
  '0011': 1962,
  '0100': 12,
  '0101': 191,
  '0110': 191,
  '0111': 760,
  '1000': 12,
  '1001': 191,
  '1010': 191,
  '1011': 760,
  '1100': 0,
  '1101': 84,
  '1110': 84,
  '1111': 448},
 '0100': {'0000': 640,
  '0001': 92,


In [110]:
list_count_l4 = {}
for i in possible_states:
    sub_list_4 = {}
    for h in possible_states:
        sub_list_4[h] = 0
    for j in possible_states:
        if list_count_l3[i][j] != 0 :
            for k in possible_states:
                if list_count[j][k] != 0:
                    sub_list_4[k] += list_count[j][k]*list_count_l3[i][j]
                else:
                    sub_list_4[k] += list_count_l3[i][k]
    list_count_l4[i] = sub_list_4

In [111]:
list_count_l4 # with length = 4

{'0000': {'0000': 63544,
  '0001': 17122,
  '0010': 17122,
  '0011': 3128,
  '0100': 17122,
  '0101': 3128,
  '0110': 3128,
  '0111': 744,
  '1000': 17122,
  '1001': 3128,
  '1010': 3128,
  '1011': 744,
  '1100': 3128,
  '1101': 744,
  '1110': 744,
  '1111': 192},
 '0001': {'0000': 17122,
  '0001': 63190,
  '0010': 3254,
  '0011': 14802,
  '0100': 3254,
  '0101': 14802,
  '0110': 748,
  '0111': 4976,
  '1000': 3254,
  '1001': 14802,
  '1010': 748,
  '1011': 4976,
  '1100': 748,
  '1101': 4976,
  '1110': 264,
  '1111': 2448},
 '0010': {'0000': 17122,
  '0001': 3254,
  '0010': 63190,
  '0011': 14802,
  '0100': 3254,
  '0101': 748,
  '0110': 14802,
  '0111': 4976,
  '1000': 3254,
  '1001': 748,
  '1010': 14802,
  '1011': 4976,
  '1100': 748,
  '1101': 264,
  '1110': 4976,
  '1111': 2448},
 '0011': {'0000': 3148,
  '0001': 15018,
  '0010': 15018,
  '0011': 43424,
  '0100': 748,
  '0101': 6092,
  '0110': 6092,
  '0111': 18640,
  '1000': 748,
  '1001': 6092,
  '1010': 6092,
  '1011': 18640,
